In [3]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re
from sklearn.feature_extraction.text import TfidfVectorizer

In [1]:
!git clone 'https://github.com/ThanhChinhBK/vietnews'

Cloning into 'vietnews'...
remote: Enumerating objects: 143827, done.
remote: Counting objects: 100% (143827/143827), done.
remote: Compressing objects: 100% (143815/143815), done.
remote: Total 143827 (delta 11), reused 143827 (delta 11), pack-reused 0
Receiving objects: 100% (143827/143827), 194.68 MiB | 16.84 MiB/s, done.
Resolving deltas: 100% (11/11), done.
Checking out files: 100% (150704/150704), done.


In [4]:
article_list = []

for file in os.listdir('/content/vietnews/data/train_tokenized'):
  #print(file)
  with open('/content/vietnews/data/train_tokenized/' + file, 'r') as f:
    article = f.read().splitlines()
    article_list.append(article[4:-1])   # leave out titles, abstracts
    f.close()

In [5]:
article_list[0]

['Góp_phần vắt_óc suy_nghĩ để cứu đội_bóng " Heo_Rừng " của Thái_Lan , có_thể nói tỉ_phú công_nghệ Elon_Musk là một trong số những người được chú_ý hàng_đầu khi ông đề_xuất chế_tạo một chiếc tàu_ngầm mini đưa vào hang chở người ra .',
 'Ông cũng liên_tục cập_nhật tình_hình , và mới_đây nhất là đích_thân đến xứ_sở Chùa vàng .',
 'Tuy_nhiên , thật tiếc khi đề_xuất của ông không được sử_dụng dù đã đưa đến tận nơi .',
 'Theo đài BBC , trả_lời phỏng_vấn mới_đây , ông Narongsak_Osotthanakorn - người đứng đầu chiến_dịch giải_cứu , nói rằng ông trân_trọng sự giúp_đỡ của tỉ_phú Musk cùng đồng_đội , nhưng " thiết_bị họ mang tới để hỗ_trợ không thực_tế với nhiệm_vụ của chúng_tôi " .',
 '" Mặc_dù thiết_bị của họ tốt và tinh_vi về mặt công_nghệ , nó không phù_hợp với nhiệm_vụ của chúng_tôi để đưa vào hang " - chỉ_huy chiến_dịch giải_cứu đội_bóng " Heo_Rừng " cho biết .',
 'Trước đó , cùng ngày , tỉ_phú Musk thông_báo trên Twitter rằng ông đã có_mặt tại hang Tham_Luang cùng với chiếc tàu_ngầm mini n

In [6]:
for i in range(len(article_list)):
  article_list[i] = list(filter(None, article_list[i]))   # Remove blank spaces
  article_list[i] = ' '.join(article_list[i])

In [7]:
article_list[0]

'Góp_phần vắt_óc suy_nghĩ để cứu đội_bóng " Heo_Rừng " của Thái_Lan , có_thể nói tỉ_phú công_nghệ Elon_Musk là một trong số những người được chú_ý hàng_đầu khi ông đề_xuất chế_tạo một chiếc tàu_ngầm mini đưa vào hang chở người ra . Ông cũng liên_tục cập_nhật tình_hình , và mới_đây nhất là đích_thân đến xứ_sở Chùa vàng . Tuy_nhiên , thật tiếc khi đề_xuất của ông không được sử_dụng dù đã đưa đến tận nơi . Theo đài BBC , trả_lời phỏng_vấn mới_đây , ông Narongsak_Osotthanakorn - người đứng đầu chiến_dịch giải_cứu , nói rằng ông trân_trọng sự giúp_đỡ của tỉ_phú Musk cùng đồng_đội , nhưng " thiết_bị họ mang tới để hỗ_trợ không thực_tế với nhiệm_vụ của chúng_tôi " . " Mặc_dù thiết_bị của họ tốt và tinh_vi về mặt công_nghệ , nó không phù_hợp với nhiệm_vụ của chúng_tôi để đưa vào hang " - chỉ_huy chiến_dịch giải_cứu đội_bóng " Heo_Rừng " cho biết . Trước đó , cùng ngày , tỉ_phú Musk thông_báo trên Twitter rằng ông đã có_mặt tại hang Tham_Luang cùng với chiếc tàu_ngầm mini nguyên_mẫu được chế_tạ

In [8]:
# Fit Tfidf
tfidf = TfidfVectorizer()
tfidf_matrix = tfidf.fit_transform(article_list)

In [9]:
print(tfidf_matrix.shape)

(105418, 238217)


Kết thúc preprocessing

In [11]:
def score_sentence(sentence, a_index_):
  # Calculate the sum tfidf score of words as sentence score
  # Take in a sentence (string), and the article's index in the corpus (for word lookup)

  sentence_score = 0
  sentence = sentence.replace(' ,', '').strip()

  # For each word in a sentence, look up its score from tfidf matrix, return 0 if lookup fails
  for i_word in sentence.split(' '):
    #print(i_word)
    sentence_score += tfidf_matrix[a_index_, tfidf.vocabulary_.get(i_word.lower(), 217377)]
  
  return sentence_score


def score_article(article, a_index_):
  # Generate a list of scores of each sentence of an article
  # Take in an article (list of sentences), and the article's index in the corpus (for word lookup)

  sentence_score_list = []
  sentence_list = article.split('.')

  # For each sentence in an article, call score_sentence and append output to sentence_score_list
  for i_sentence in sentence_list:
    sentence_score = score_sentence(i_sentence, a_index_)
    sentence_score_list.append(sentence_score)

  return sentence_score_list, sentence_list


def generate_summary(sentence_score_list, sentence_list, alpha=1.15):
  # Generate a summary from sentences that have score alpha times larger than the article' sentence mean score
  # Take in a list of sentence scores, list of sentences, thresholding multiplier alpha

  sentence_score_array = np.asarray(sentence_score_list)
  mean_score = np.mean(sentence_score_array)
  sentence_list_arr = np.asarray(sentence_list, dtype=object)
  summary = sentence_list_arr[sentence_score_array > alpha*mean_score]
  summary = ' '.join(summary)

  return summary


In [30]:
# Generate summary for 50 articles
a_index_ = 0
summary_list = []

for i_article in article_list[:50]:
  #print(a_index_)
  #print(i_article)
  sentence_score_list, sentence_list = score_article(i_article, a_index_)
  summary = generate_summary(sentence_score_list, sentence_list)
  summary_list.append(summary)
  a_index_ += 1

In [ ]:
#np.argmin(np.sum(tfidf_matrix, axis=0))

217377

In [31]:
summary_list[:5]

['Góp_phần vắt_óc suy_nghĩ để cứu đội_bóng " Heo_Rừng " của Thái_Lan , có_thể nói tỉ_phú công_nghệ Elon_Musk là một trong số những người được chú_ý hàng_đầu khi ông đề_xuất chế_tạo một chiếc tàu_ngầm mini đưa vào hang chở người ra   Theo đài BBC , trả_lời phỏng_vấn mới_đây , ông Narongsak_Osotthanakorn - người đứng đầu chiến_dịch giải_cứu , nói rằng ông trân_trọng sự giúp_đỡ của tỉ_phú Musk cùng đồng_đội , nhưng " thiết_bị họ mang tới để hỗ_trợ không thực_tế với nhiệm_vụ của chúng_tôi "   Trước đó , cùng ngày , tỉ_phú Musk thông_báo trên Twitter rằng ông đã có_mặt tại hang Tham_Luang cùng với chiếc tàu_ngầm mini nguyên_mẫu được chế_tạo để đưa 4 cầu_thủ nhí và 1 huấn_luyện_viên vẫn còn mắc_kẹt trong hang ra ngoài   Tướng Werachon_Sukondhapatipak - người_phát_ngôn của Thủ_tướng Prayuth_Chan - o - cha ngày 10-7 cho_biết Thủ_tướng Thái_Lan rất cảm_kích sáng_kiến của tỉ_phú Elon_Musk để giải_cứu đội_bóng " Heo_Rừng " và ông cực_kỳ cảm_động vì ông Musk đã đích_thân tới Chiang_Rai để giúp_đỡ 

Generate summary for 1 article

In [19]:
text = article_list[0]

In [23]:
sentence_score_list, sentence_list = score_article(text, 0)
summary = generate_summary(sentence_score_list, sentence_list)

In [24]:
print(sentence_score_list)

[3.224226572838488, 0.5496848818895798, 0.6462591822038841, 2.321100918478459, 1.6626012719874939, 2.9014376141126155, 0.45680082034860303, 1.0318428104309212, 3.4153403064442553, 1.851687274739122, 1.1997173754962103, 2.430584418474682, 0.8600666451147057, 0.8684320615089924, 0.4853442476031815, 1.4122993860539312, 1.0743042145067372, 2.1358830844275154, 0.9658309311354398, 1.7644417288825582, 1.2331143787271774, 1.7379031508191103, 2.656990506623013, 1.2485257862888153, 1.1098945371549294, 0.7367191478168912, 2.013132050247676, 1.640991389616884, 2.009908483252617, 0.7549169975696446, 1.7517324557304546, 1.6622635875451823, 0.0736614735479328, 0.7345372726653336, 0.0]


In [26]:
generate_summary(sentence_score_array, sentence_list)

'Góp_phần vắt_óc suy_nghĩ để cứu đội_bóng " Heo_Rừng " của Thái_Lan , có_thể nói tỉ_phú công_nghệ Elon_Musk là một trong số những người được chú_ý hàng_đầu khi ông đề_xuất chế_tạo một chiếc tàu_ngầm mini đưa vào hang chở người ra   Theo đài BBC , trả_lời phỏng_vấn mới_đây , ông Narongsak_Osotthanakorn - người đứng đầu chiến_dịch giải_cứu , nói rằng ông trân_trọng sự giúp_đỡ của tỉ_phú Musk cùng đồng_đội , nhưng " thiết_bị họ mang tới để hỗ_trợ không thực_tế với nhiệm_vụ của chúng_tôi "   Trước đó , cùng ngày , tỉ_phú Musk thông_báo trên Twitter rằng ông đã có_mặt tại hang Tham_Luang cùng với chiếc tàu_ngầm mini nguyên_mẫu được chế_tạo để đưa 4 cầu_thủ nhí và 1 huấn_luyện_viên vẫn còn mắc_kẹt trong hang ra ngoài   Tướng Werachon_Sukondhapatipak - người_phát_ngôn của Thủ_tướng Prayuth_Chan - o - cha ngày 10-7 cho_biết Thủ_tướng Thái_Lan rất cảm_kích sáng_kiến của tỉ_phú Elon_Musk để giải_cứu đội_bóng " Heo_Rừng " và ông cực_kỳ cảm_động vì ông Musk đã đích_thân tới Chiang_Rai để giúp_đỡ  

In [27]:
len(article_list[0])

4705

In [28]:
len(summary)

2492